In [1]:
import pandas as pd
import plotly.express as px
import os 


In [2]:
test_parquet=os.path.join(os.getcwd(), 'data', 'asl-signs', 'train_landmark_files','37055', "1003109377.parquet") #this is on the dataset
df= pd.read_parquet(test_parquet) #this is on the dataset

In [3]:
df.head()

,frame,row_id,type,landmark_index,x,y,z
0,14,14-face-0,face,0,0.432380,0.375417,-0.044000
1,14,14-face-1,face,1,0.437980,0.349091,-0.070176
2,14,14-face-2,face,2,0.429624,0.360922,-0.040331
3,14,14-face-3,face,3,0.430260,0.315678,-0.049688
4,14,14-face-4,face,4,0.441155,0.338613,-0.073218


In [4]:
df.columns

Index(['frame', 'row_id', 'type', 'landmark_index', 'x', 'y', 'z'], dtype='object')

In [5]:
df

,frame,row_id,type,landmark_index,x,y,z
0,14,14-face-0,face,0,0.432380,0.375417,-0.044000
1,14,14-face-1,face,1,0.437980,0.349091,-0.070176
2,14,14-face-2,face,2,0.429624,0.360922,-0.040331
3,14,14-face-3,face,3,0.430260,0.315678,-0.049688
4,14,14-face-4,face,4,0.441155,0.338613,-0.073218
...,...,...,...,...,...,...,...
12484,36,36-right_hand-16,right_hand,16,NaN,NaN,NaN
12485,36,36-right_hand-17,right_hand,17,NaN,NaN,NaN
12486,36,36-right_hand-18,right_hand,18,NaN,NaN,NaN
12487,36,36-right_hand-19,right_hand,19,NaN,NaN,NaN


In [6]:
px.scatter_3d(df, x="x", y="y", z="z", color="type" , opacity=0.5, width=800, height=800).show()

In [7]:
df2=pd.read_parquet("2d_landmarks.parquet") #rgb image landmarks
df2.columns

Index(['type', 'landmark_index', 'x', 'y', 'z', 'frame'], dtype='object')

In [8]:
df2

,type,landmark_index,x,y,z,frame
0,face,0,0.352590,0.255115,6.659,1
1,face,1,0.358590,0.225441,5.802,1
2,face,2,0.352623,0.235474,6.234,1
3,face,3,0.351185,0.197310,5.140,1
4,face,4,0.359609,0.216578,5.376,1
...,...,...,...,...,...,...
10312,right_hand,16,NaN,NaN,NaN,19
10313,right_hand,17,NaN,NaN,NaN,19
10314,right_hand,18,NaN,NaN,NaN,19
10315,right_hand,19,NaN,NaN,NaN,19


In [9]:
df2['z'].unique()

array([6.6590004 , 5.80200005, 6.23400021, 5.14000034, 5.3760004 ,
       5.20800018, 4.84300041, 4.80300045, 4.57800007, 4.40600014,
       3.89300013, 6.62100029, 6.54700041, 6.58400011, 6.6970005 ,
       5.94900036, 6.04100037, 4.24600029, 4.88300037, 4.90300035,
       4.96600008, 4.56000042, 4.59600019, 5.00800037, 6.51100016,
       4.74500036, 4.82300043, 5.55400038, 5.77300024, 5.42600012,
       4.38900042, 5.6340003 , 5.23200035, 4.30800009, 3.9860003 ,
       4.43900013, 4.6500001 , 6.43900013, 6.20100021, 5.97900009,
       4.26100016, 5.71700048, 4.23100042, 3.85500026, 4.14100027,
       4.04100037, 4.32400036, 5.88900042, 6.13600016, 5.86000013,
       6.10400009, 5.32700014, 4.05500031, 4.27700043, 4.1260004 ,
       3.86800027, 5.16300011, 4.86300039, 5.58100033, 5.25500011,
       5.30300045, 5.09500027, 4.98700047, 5.45100021, 4.61400032,
       5.35100031, 4.92400026, 5.50200033, 5.52800035, 4.5250001 ,
       6.0720005 , 6.85300016, 7.19000053, 4.76400042, 4.72500

In [10]:
px.scatter_3d(df2, x="x", y="y", z="z", color="type" , opacity=0.5, width=800, height=800).show()

In [11]:
df3= pd.read_parquet("3d_landmarks.parquet") #rgb image landmarks

In [12]:
df3

,frame,type,landmark_index,x,y,z
0,1,pose,0,-0.779627,-0.573723,2.587
1,1,pose,1,-0.516047,-0.565620,2.078
2,1,pose,2,-0.800289,-0.641170,2.499
3,1,pose,3,-0.215601,-0.567442,2.220
4,1,pose,5,0.054267,0.004651,0.705
...,...,...,...,...,...,...
734,9,left_hand,19,-0.220240,0.045064,0.973
735,9,left_hand,20,-0.209285,0.055183,0.970
736,9,right_hand,0,-1.003345,0.899424,1.970
737,9,right_hand,3,-0.578615,0.545179,0.990


In [13]:
df3.columns

Index(['frame', 'type', 'landmark_index', 'x', 'y', 'z'], dtype='object')

In [14]:
px.scatter_3d(df3, x="x", y="y", z="z", color="type" , opacity=0.5, width=800, height=800).show()

In [15]:
print(df.shape, df2.shape, df3.shape)

(12489, 7) (10317, 6) (739, 6)
